## [Tutorial] Electra Domain Adaptation Tutorial With Huggingface

### 참고사항
* 단계별 상세 설명은 [Huggingface로 ELECTRA 학습하기 : Domain Adaptation](https://yangoos57.github.io/blog/DeepLearning/paper/Electra/electra/) 참고

* 구동환경

  ```python
    torch == 1.12.1
    pandas == 1.4.3
    transformers == 4.20.1
    datasets == 2.8.0
  ```


### 1. Electra Model 불러오기

* Generator는 ElectraForMaskedLM로 불러오고, Descriminator은 ElectraForPreTraining로 불러와야함.

* ElectraForMaskedLM는 Mask 토큰에 들어갈 단어를 예측하는 기능, ElectraForPreTraining는 문장 내 토큰의 진위여부를 판별하는 기능을 수행함.

* [monologg님의 KoELECTRA](https://github.com/monologg/KoELECTRA)를 베이스 모델로 활용


In [1]:
from transformers import ElectraForPreTraining, ElectraTokenizer, ElectraForMaskedLM,ElectraForSequenceClassification

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

generator = ElectraForMaskedLM.from_pretrained('monologg/koelectra-base-v3-generator')
discriminator = ElectraForPreTraining.from_pretrained("monologg/koelectra-base-v3-discriminator")


### 2. Huggingface의 Datasets 라이브러리로 데이터 불러오기
* Huggingface의 Trainer로 모델을 학습할 예정이라면 Datasets으로 학습 자료를 불러오는 것을 추천

* pytorch의 Dataset으로 Trainer를 사용할 수 있으나 경험 상 디버깅이 상당히 번거로움.
* Trainer와 연동성이 보장된 Dataset은 간편하게 데이터를 활용할 수 있음

In [2]:
from datasets import load_dataset

train = load_dataset('csv',data_files='data/book_train_128.csv')
validation = load_dataset('csv',data_files='data/book_validation_128.csv')

Using custom data configuration default-189a9ba3a9938559
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-189a9ba3a9938559/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-5db4eff94cdf2f03
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-5db4eff94cdf2f03/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

### 3. 데이터 토크나이징
* Trainer에 활용하기 위해선 데이터에 대한 토크나이징을 수행해야함.

* Datasets에서 제공하는 map 함수를 활용하면 간편하게 토크나이징이 가능함.

In [3]:
def tokenize_function(examples):
    return tokenizer(examples['sen'], max_length=128, padding=True, truncation=True)

train_data_set = train['train'].map(tokenize_function)
validation_data_set = validation['train'].map(tokenize_function)

Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-189a9ba3a9938559/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-9bedb7d002b69da2.arrow
Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-5db4eff94cdf2f03/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3aec615c5e921d3e.arrow


### 4. Generator와 Descriminator 학습을 위한 Electra Model 설계

- Electra-pytorch 라이브러리를 Transformers 라이브러리에서 활용할 수 있도록 일부 수정하였음.

- Electra-pytorh 원본 github 주소 : https://github.com/lucidrains/electra-pytorch

- 해당 모델은 아래의 3단계를 수행하기 위해 설계되었음
    - 1단계 : input data masking
    - 2단계 : Generator 학습 및 fake sentence 생성
    - 3단계 : Discriminator 학습



In [4]:
import math
from functools import reduce
from collections import namedtuple

import torch
from torch import nn
import torch.nn.functional as F

# constants

Results = namedtuple(
    "Results",
    [
        "loss",
        "mlm_loss",
        "disc_loss",
        "gen_acc",
        "disc_acc",
        "disc_labels",
        "disc_predictions",
        "origin",
        "disc",
    ],
)

# 모델 내부에서 활용되는 함수 정의


def log(t, eps=1e-9):
    return torch.log(t + eps)


def gumbel_noise(t):
    noise = torch.zeros_like(t).uniform_(0, 1)
    return -log(-log(noise))


def gumbel_sample(t, temperature=1.0):
    return ((t / temperature) + gumbel_noise(t)).argmax(dim=-1)


def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob


def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask


def get_mask_subset_with_prob(mask, prob):
    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)

    num_tokens = mask.sum(dim=-1, keepdim=True)
    mask_excess = mask.cumsum(dim=-1) > (num_tokens * prob).ceil()
    mask_excess = mask_excess[:, :max_masked]

    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)
    _, sampled_indices = rand.topk(max_masked, dim=-1)
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)

    new_mask = torch.zeros((batch, seq_len + 1), device=device)
    new_mask.scatter_(-1, sampled_indices, 1)
    return new_mask[:, 1:].bool()


# main electra class


class Electra(nn.Module):
    def __init__(
        self,
        generator,
        discriminator,
        tokenizer,
        *,
        num_tokens=35000,
        mask_prob=0.15,
        replace_prob=0.85,
        mask_token_id=4,
        pad_token_id=0,
        mask_ignore_token_ids=[2, 3],
        disc_weight=50.0,
        gen_weight=1.0,
        temperature=1.0,
    ):
        super().__init__()

        """
        num_tokens: 모델 vocab_size
        mask_prob: 토큰 중 [MASK] 토큰으로 대체되는 비율
        replace_prop:  토큰 중 [MASK] 토큰으로 대체되는 비율(?????)
        mask_token_i: [MASK] Token id
        pad_token_i: [PAD] Token id
        mask_ignore_token_id: [CLS],[SEP] Token id
        disc_weigh: discriminator loss의 Weight 조정을 위한 값
        gen_weigh: generator loss의 Weight 조정을 위한 값
        temperature: gumbel_distribution에 활용되는 arg, 값이 높을수록 모집단 분포와 유사한 sampling 수행
        """

        self.generator = generator
        self.discriminator = discriminator
        self.tokenizer = tokenizer

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

        # sampling temperature
        self.temperature = temperature

        # loss weights
        self.disc_weight = disc_weight
        self.gen_weight = gen_weight

    def forward(self, input_ids, **kwargs):

        try:
            input = input_ids["input_ids"]
        except:
            input = input_ids

        # ------ 1단계 Input Data Masking --------#

        """
        - Generator는 Bert와 구조도 동일하고 학습하는 방법도 동일함. 

        - Generator 학습을 위해선 [Masked] 토큰이 필요하므로 input data를 Masking하는 과정이 필요함.

        """

        replace_prob = prob_mask_like(input, self.replace_prob)

        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # set inverse of mask to padding tokens for labels
        gen_labels = input.masked_fill(~mask, self.pad_token_id)

        # clone the mask, for potential modification if random tokens are involved
        # not to be mistakened for the mask above, which is for all tokens, whether not replaced nor replaced with random tokens
        masking_mask = mask.clone()

        # [mask] input
        masked_input = masked_input.masked_fill(
            masking_mask * replace_prob, self.mask_token_id
        )

        # ------ 2단계 Masking 된 문장을 Generator가 학습하고 가짜 Token을 생성 --------#

        """
        - Generator를 학습하여 MLM_loss 계산(combined_loss 계산에 활용)
        - Generator에서 예측한 문장을 Discriminator 학습에 활용
        - ex) 원본 문장 :  특히 안드로이드 플랫폼 기반의 (웹)앱과 (하이)브드리앱에 초점을 맞추고 있다
              가짜 문장 :  특히 안드로이드 플랫폼 기반의 (마이크로)앱과 (이)브드리앱에 초점을 맞추고 있다

        """

        # get generator output and get mlm loss(수정)
        logits = self.generator(masked_input, **kwargs).logits

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2), gen_labels, ignore_index=self.pad_token_id
        )

        # use mask from before to select logits that need sampling
        sample_logits = logits[mask_indices]

        # sample
        sampled = gumbel_sample(sample_logits, temperature=self.temperature)

        # scatter the sampled values back to the input
        disc_input = input.clone()
        disc_input[mask_indices] = sampled.detach()

        # generate discriminator labels, with replaced as True and original as False
        disc_labels = (input != disc_input).float().detach()

        # ------ 3단계 가짜 Token의 진위여부를 Discriminator가 판단하는 단계 --------#

        """
        - 가짜 문장을 학습해 개별 토큰에 대해 진위여부를 판단
        - 진짜 token이라 판단하면 0, 가짜 토큰이라 판단하면 1을 부여
        - 정답과 비교해 disc_loss를 계산(combined_loss 계산에 활용)
        - combined_loss : 학습의 최종 loss임. 모델은 combined_loss의 최솟값을 얻기 위한 방식으로 학습 진행
        """

        # get discriminator predictions of replaced / original
        non_padded_indices = torch.nonzero(input != self.pad_token_id, as_tuple=True)

        # get discriminator output and binary cross entropy loss
        disc_logits = self.discriminator(disc_input, **kwargs).logits
        disc_logits_reshape = disc_logits.reshape_as(disc_labels)

        disc_loss = F.binary_cross_entropy_with_logits(
            disc_logits_reshape[non_padded_indices], disc_labels[non_padded_indices]
        )

        # combined loss 계산
        # disc_weight을 50으로 주는 이유는 discriminator의 task가 복잡하지 않기 떄문임.
        # mlm loss의 경우 vocab_size(=35000) 만큼의 loos 계산을 수행하지만
        # disc_loss의 경우 src_token_len 만큼의 loss 계산을 수행한만큼
        # loss 값에 큰 차이가 발생함. disc_weight은 이를 보완하는 weight임.
        combined_loss = self.gen_weight * mlm_loss + self.disc_weight * disc_loss

        # ------ 모델 성능 및 학습 과정을 추적하기 위한 지표(Metrics) 설계 --------#

        with torch.no_grad():
            # gen mask 예측
            gen_predictions = torch.argmax(logits, dim=-1)

            # fake token 진위 예측
            disc_predictions = torch.round(
                (torch.sign(disc_logits_reshape) + 1.0) * 0.5
            )
            # generator_accuracy
            gen_acc = (gen_labels[mask] == gen_predictions[mask]).float().mean()

            # discriminator_accuracy
            disc_acc = (
                0.5 * (disc_labels[mask] == disc_predictions[mask]).float().mean()
                + 0.5 * (disc_labels[~mask] == disc_predictions[~mask]).float().mean()
            )

        return Results(
            combined_loss,
            mlm_loss,
            disc_loss,
            gen_acc,
            disc_acc,
            disc_labels,
            disc_predictions,
            input,
            disc_input,
        )


### 5. 모델 불러오기 

In [5]:
device = 'cpu'

model = Electra(generator=generator,discriminator=discriminator,tokenizer=tokenizer)

### 6. Trainer 기타 기능 설정 및 학습


#### ✓ 훈련 옵션 설정(선택사항)

* 훈련에 사용되는 모든 arguments를 `TrainingArguments`를 통해 조정할 수 있음

* `logging_stetps`는 {loss,learning_rate,epoch} 정보를 몇번의 step 간격으로 수행해야할지 설정
* `evaluation_strategy`는 training 중 evaluation을 어느 때 실행해야할지 설정 'epoch'와 'step'이 있음. evaluation_strategy를 설정하지 않으면 학습 중 evaluation을 진행하지 않음.


#### ✓ Input data 가공을 위한 Data collater 설정
* Data callter은 학습 목적에 맞게 input data를 가공하는 방법을 설정

* `DataCollatorForLanguageModeling`는 Input_data에 [MASK]를 포함하도록 가공하는 collater임. 따라서 Bert 모델 학습에 필히 설정해야함.

* Transformers는 `DataCollatorForLanguageModeling` 외에도 여러 학습 방법에 맞게 데이터를 가공하는 collater를 제공 (`DataCollatorWithPadding`, `DataCollatorForTokenClassification` 등)



#### ✓ Callback 정의하기(선택사항)

> callback에 대한 상세한 설명은 ____ 참고

* callback은 학습 중 Trainer가 추가로 수행해야하는 Task를 정의함.

* 미리 정의된 callback을 사용하거나 아래 코드와 같이 커스텀하여 사용할 수 있음

* 아래의 `myCallback`은 100번째 Step마다 현재 epoch와 step을 출력하는 Task를 정의함.


#### ✓ Custom Trainer 만들기(선택사항)

* Trainer 내부 함수를 목적에 맞게 변경할 수 있음.

* Trainer를 커스터마이징하면 아래의 예시처럼 모델 학습 경과를 시각화 할 수 있음.

```python 
    0번째 epoch 진행 중 ------- 20번째 step 결과
    input 문장 : [MASK]이 출간된지 꽤 됬다고 생각하는데 실습하는데 전혀 [MASK]없습니다
    output 문장 : [책]이 출간된지 꽤 됬다고 생각하는데 실습하는데 전혀 [문제]없습니다
```

* Trainer 내부의 `compute_loss` 함수를 활용하면 input_data와 모델 학습 결과인 output_data에 접근할 수 있음

> 해당 매서드를 callback으로 구현하기에는 callback이 input_data와 output_data에 접근하기 까다롭기 때문에 
>
> Trainer를 커스터마이징 하는 방법을 추천


#### ✓ Trainer 정의 및 학습 시작

* 지금까지 설정한 옵션, 데이터셋을 Trainer의 args로 활용

* 이후 train() 매서드를 통해 학습 시작

* Trainer는 매 500회 step 이후 학습된 모델을 저장하며, 학습이 중간에 중단되더라도 trainer('폴더 경로')를 통해 중단된 부분부터 새롭게 학습이 가능함.

In [ ]:
from transformers import TrainingArguments, TrainerCallback,Trainer,DataCollatorForLanguageModeling
from IPython.display import display, HTML
import pandas as pd

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=10,
    eval_steps=100,
    num_train_epochs=2,
    # evaluation_strategy='epoch'
    )

class myCallback(TrainerCallback):

    def on_step_begin(self, args, state, control, logs=None, **kwargs):

        # 함수 이름.. 언제 시작할지
        # log는 설정할 때마다
        # arg,state,control은 참고할 수 있는 attribute의 경우임.
        # 근데 내가 필요한건 input
        if state.global_step % args.logging_steps == 0:
            print("")
            print(
                f"{int(state.epoch)}번째 epoch 진행 중 ------- {state.global_step}번째 step 결과"
            )


class customtrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    ############# 내용 추가
    def step_check(self):
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        # ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            with torch.no_grad() :
                
                origin_id = outputs.origin[0].tolist()
                pad_idx = origin_id.index(0) if 0 in origin_id else None

                origin_id = origin_id[:pad_idx]
                disc_id = outputs.disc[0].tolist()[:pad_idx]

                origin_tokens = tokenizer.convert_ids_to_tokens(origin_id)
                
                # print 용도 
                origin_tokens_for_print = origin_tokens.copy()

                disc_tokens = tokenizer.convert_ids_to_tokens(disc_id)


                mask_idx = (outputs.disc_labels[0] == 1).nonzero(as_tuple = True)[0].tolist()

                # 가짜 토큰 표시    
                for i in mask_idx:
                    origin_tokens_for_print[i] = "(" + origin_tokens_for_print[i] + ")"
                    disc_tokens[i] = "(" + disc_tokens[i] + ")"

                # 가짜 토큰 index
                fake_idx = (outputs.disc_labels[0][:pad_idx] == 1).nonzero(as_tuple=True)[0].tolist()
                prd_idx = (outputs.disc_predictions[0][:pad_idx] == 1).nonzero(as_tuple=True)[0].tolist()


                # l2 = 가짜 토큰을 진짜 토큰으로 판단한 경우(오답)
                l2 = []
                for i in fake_idx :
                    l2.append([i,origin_tokens[i],disc_tokens[i]])

                # l3 = 진짜 토큰을 가짜 토큰으로 판단한 경우(오답)
                l3 = []
                for i in prd_idx :
                    l3.append([i,origin_tokens[i],disc_tokens[i]])

                # l1 = 가짜 토큰을 가짜토큰으로 판단한 경우(정답)
                l1 = []
                x = l2.copy()
                y = l3.copy()
                for i in x :
                    for j in y : 
                        if i == j :
                            l2.pop(l2.index(i))
                            l3.pop(l3.index(j))
                            l1.append(i)
                            break

                # l1 = 가짜 토큰을 가짜토큰으로 판단한 경우(정답)
                l1 = list(map(lambda x : x+['fake']+['fake']+['O'],l1))

                # l2 = 가짜 토큰을 진짜 토큰으로 판단한 경우(오답)
                l2 = list(map(lambda x : x+['fake']+['-']+['X'],l2))

                # l3 = 진짜 토큰을 가짜 토큰으로 판단한 경우(오답)
                l3 = list(map(lambda x : x+['-']+['fake']+['X'],l3))

                x = pd.DataFrame(l1+l2+l3)

                if len(x) != 0 :
                    x.columns = ['문장 위치','실제 토큰','(가짜)토큰','실제','예측','정답']


                ### --- 
                print('')
                print('')
                print('원본 문장 : ',tokenizer.convert_tokens_to_string(origin_tokens_for_print[1:-1]))
                print('가짜 문장 : ',tokenizer.convert_tokens_to_string(disc_tokens[1:-1]))
                print('')
                print('')
                print(f'{len(origin_id)}개 토큰 중 {len(l2+l3)}개 예측 실패 -------- {len(fake_idx)}개 가짜 토큰 중 {len(l1)}개 판별')
                display(HTML(x.to_html()))
                print(f'Combined Loss {round(outputs.loss.item(),3)} -- Generator Loss : {round(outputs.mlm_loss.item(),3)} -- Discriminator Loss : {round(outputs.disc_loss.item(),3)}')
                

        return (loss, outputs) if return_outputs else loss

trainer = customtrainer(
    model=model.to(device),
    train_dataset=train_data_set,
    eval_dataset=validation_data_set,
    args=training_args,
    tokenizer=tokenizer,
    callbacks=[myCallback],
)

trainer.train()



### 학습한 모델 저장하기

### 학습이 끝난 직후 모델 저장하기 

In [ ]:
# 향후 추가적인 학습을 하고싶은 경우 generator, discriminator 전체 저장
trainer.model.generator.save_pretrained('model/generator')
trainer.model.discriminator.save_pretrained('model/discriminator')

# 향후 추가 학습 계획이 없고 학습한 모델을 활용해 Fine-tuning만 하려는 경우, basemodel만 저장
trainer.model.discriminator.base_model.save_pretrained('model/discriminator')


### 학습 중 저장된 check_point에서 모델을 저장하고 싶은 경우

In [ ]:
# check point에 저장된 모델 불러오기
model = Electra(generator=generator,discriminator=discriminator,tokenizer=tokenizer)
path = torch.load('model/checkpoint-100/pytorch_model.bin',map_location=torch.device('cpu'))
model.load_state_dict(state_dict=path)


# 향후 추가적인 학습을 하고싶은 경우 generator, discriminator 전체 저장
model.generator.save_pretrained('model/generator')
model.discriminator.save_pretrained('model/discriminator')

# 향후 추가 학습 계획이 없고 학습한 모델을 활용해 Fine-tuning만 하려는 경우, basemodel만 저장
model.discriminator.base_model.save_pretrained('model/discriminator')

### 실제로 학습된 모델을 불러온 것인지 확인하고 싶은 경우(옵션)

In [32]:
# 학습하기 전 초기 모델 불러오기
old_discriminator = ElectraForPreTraining.from_pretrained("monologg/koelectra-base-v3-discriminator")

# 학습 된 모델 불러오기
model = Electra(generator=generator,discriminator=discriminator,tokenizer=tokenizer)
path = torch.load('model/checkpoint-100/pytorch_model.bin',map_location=torch.device('cpu'))
model.load_state_dict(state_dict=path)

new_discriminator = model.discriminator


# 불러온 모델 간 embedding 차이를 확인하여 실제 학습한 모델을 불러온 것인지 체크
old_param = list(old_discriminator.parameters())
new_param = list(new_discriminator.parameters())

not_updated_params = []
for i in range(len(old_param)) :
    x = old_param[i] - new_param[i]
    if torch.sum(x.reshape(-1)).item() == 0 :
        not_updated_params.append(i)

print(not_updated_params)

### 저장한 모델 불러오기 

In [ ]:
from transformers import ElectraForPreTraining, ElectraTokenizer, ElectraForMaskedLM,ElectraForSequenceClassification

generator = ElectraForMaskedLM.from_pretrained('model/generator')

discriminator = ElectraForPreTraining.from_pretrained('model/discriminator')

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

